In [1]:
!pip install ratelimit

  Stored in directory: /root/.cache/pip/wheels/05/d9/82/3c6044cf1a54aab9151612458446d9b17a38416869e1b1d9b8
Successfully built ratelimit


In [0]:
import pandas as pd
import requests
import json
import numpy as np 
from ratelimit import limits, sleep_and_retry

In [0]:
api_key = ''
api_url = 'https://api.themoviedb.org/3/'
SearchPeople = pd.DataFrame.from_dict({"data":[]})
SearchMovies = pd.DataFrame.from_dict({"data":[]})

IDPeople = pd.DataFrame.from_dict({"data":[]})
IDMovies = pd.DataFrame.from_dict({"data":[]})

MovieGenres = [] 

In [0]:
# The official limit is 
# 40 request per 10 seconds

TEN_SECONDS = 10
REQUESTS = 39

@sleep_and_retry
@limits(calls=REQUESTS, period=TEN_SECONDS)
def call_api(url):
    response = requests.get(url)
    if response.status_code == 429:
        raise Exception('API response: {}'.format(response.status_code))
    return response
  
MovieGenres = call_api(api_url+"genre/movie/list?api_key="+api_key).json()['genres']

In [5]:
MovieGenres

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 80, 'name': 'Crime'},
 {'id': 99, 'name': 'Documentary'},
 {'id': 18, 'name': 'Drama'},
 {'id': 10751, 'name': 'Family'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 36, 'name': 'History'},
 {'id': 27, 'name': 'Horror'},
 {'id': 10402, 'name': 'Music'},
 {'id': 9648, 'name': 'Mystery'},
 {'id': 10749, 'name': 'Romance'},
 {'id': 878, 'name': 'Science Fiction'},
 {'id': 10770, 'name': 'TV Movie'},
 {'id': 53, 'name': 'Thriller'},
 {'id': 10752, 'name': 'War'},
 {'id': 37, 'name': 'Western'}]

In [0]:
# Search a person
def searchPerson(person):
  global SearchPeople
  try:
    return SearchPeople.loc[person,"data"]
  except: 
    url = api_url+"search/person?api_key="+api_key+"&query=\""+person+"\""
    response = call_api(url)
    data = response.json()
    SearchPeople = SearchPeople.append(pd.Series({'data':data},name=person))
    return data
# Search a movie
def searchMovie(movie):
  global SearchMovies
  try:
    return SearchMovies.loc[movie,"data"]
  except:
    page = 1
    data = {}
    url = api_url+"search/movie?api_key="+api_key+"&query=\""+movie+"\"&page="+str(page)
    response = call_api(url).json()
    data[page] = response["results"]
    total_pages = response["total_pages"]
    page += 1
    
    while(page <= total_pages):
      url = api_url+"search/movie?api_key="+api_key+"&query=\""+movie+"\"&page="+str(page)
      response = call_api(url).json()
      data[page] = response["results"]
      page += 1
      
    
    results = [result for page in data.values() for result in page]
    SearchMovies = SearchMovies.append(pd.Series({'data':results},name=movie))
    return results

In [0]:
# Person and movie credits
def getPerson(person_id):
  global IDPeople
  try:
    return IDPeople.loc[person_id,"data"]
  except: 
    url = api_url+"person/"+str(person_id)+"?api_key="+api_key+"&append_to_response=movie_credits"
    response = call_api(url)
    data = response.json()
    IDPeople = IDPeople.append(pd.Series({'data':data},name=person_id))
    return data

# Movie and credits
def getMovie(movie_id):
  global IDMovies
  try:
    return IDMovies.loc[movie_id,"data"]
  except: 
    url = api_url+"movie/"+str(movie_id)+"?api_key="+api_key+"&append_to_response=credits"
    response = call_api(url)
    data = response.json()
    IDMovies = IDMovies.append(pd.Series({'data':data},name=movie_id))
    return data

In [0]:
# Get the genre object from the genre_id
def getGenre(genre_id):
  return [genre for genre in MovieGenres if genre["id"] == genre_id][0]

# get the movies with a genre from a list of movies
def getMoviesWithGenre(genre_id, movies):
  return [movie for movie in movies if getGenre(genre_id) in movie["genres"]]

In [0]:
# List of person with a job in a crew
def getJobFrom(job, crew):
  return [value for value in crew if value["job"] == job]

# List of works of someone in a crew or cast
def getNameFrom(name, crew):
  return [value for value in crew if value["name"] == name]

# Crew of a movie
def getCrew(movie_id):
  movie = getMovie(movie_id)
  try:
    return movie["credits"]["crew"]
  except:
    return movie["status_message"]  
  
# Cast of a movie
def getCast(movie_id):
  movie = getMovie(movie_id)
  try:
    return movie["credits"]["cast"]
  except:
    return movie["status_message"]

In [0]:
# Get the roles of someone in a list of movies
def getRoles(name, movies):
  return [(movie["title"],cast["character"]) for movie in movies for cast in movie["credits"]["cast"] if cast["name"] == name] 

# Get the people that has worked as a character in a list of movies
def getCharacter(char, movies):
  return [(movie["title"],cast["name"], cast["character"]) for movie in movies for cast in movie["credits"]["cast"] if char in cast["character"]] 

# Get the movies that has worked as a character in a list of movies
def getCharacter(char, movies):
  return [(movie["title"],cast["name"], cast["character"]) for movie in movies for cast in movie["credits"]["cast"] if char in cast["character"]] 

In [0]:
crew = getPerson(searchPerson("Stan Lee")["results"][0]["id"])["movie_credits"]["crew"]
movies_ids = [movie["id"] for movie in crew]
movies_ids = np.unique(np.array(movies_ids))
movies = [getMovie(movie) for movie in movies_ids]

In [13]:
getRoles("Chris Evans", movies)

[('Captain America: The First Avenger', 'Steve Rogers / Captain America'),
 ('Fantastic Four: Rise of the Silver Surfer', 'Johnny Storm / Human Torch'),
 ('Fantastic Four', 'Johnny Storm / Human Torch'),
 ('The Avengers', 'Steve Rogers / Captain America'),
 ('Thor: The Dark World', 'Loki as Captain America (uncredited)'),
 ('Avengers: Age of Ultron', 'Steve Rogers / Captain America'),
 ('Captain America: The Winter Soldier', 'Steve Rogers / Captain America'),
 ('Ant-Man', 'Steve Rogers / Captain America (uncredited)'),
 ('Captain America: Civil War', 'Steve Rogers / Captain America'),
 ('Avengers: Endgame', 'Steve Rogers / Captain America'),
 ('Avengers: Infinity War', 'Steve Rogers / Captain America'),
 ('Captain Marvel', 'Steve Rogers / Captain America (uncredited)'),
 ('Spider-Man: Homecoming', 'Steve Rogers / Captain America')]

In [0]:
batman_movies_ids = [movie["id"] for movie in searchMovie("Batman")]
batman_movies_ids = np.unique(np.array(batman_movies_ids))
batman_movies = [getMovie(movie) for movie in batman_movies_ids]

batman_cast = [(cast["name"]) for movie in batman_movies for cast in movie["credits"]["cast"]] 


In [0]:
batman_animation = getMoviesWithGenre(16,batman_movies)

In [17]:
[movie["title"] for movie in batman_animation]

['Batman: Gotham Knight',
 'Batman: Mask of the Phantasm',
 'Batman & Mr. Freeze: SubZero',
 'Batman Beyond: Return of the Joker',
 "The Batman Superman Movie: World's Finest",
 'The Batman vs. Dracula',
 'Batman: Mystery of the Batwoman',
 'Superman/Batman: Public Enemies',
 'Batman: Under the Red Hood',
 'Superman/Batman: Apocalypse',
 'Batman Beyond: The Movie',
 'Batman: Year One',
 'Batman: The Dark Knight Returns, Part 1',
 'Batman: The Dark Knight Returns, Part 2',
 'Lego Batman: The Movie - DC Super Heroes Unite',
 'Black Star and the Golden Bat',
 'Batman: A Gotham Fairytale',
 'Batman: Assault on Arkham',
 'Son of Batman',
 'Batman vs. The Terminator',
 'Batman: Strange Days',
 'Batman Beyond',
 'Scooby-Doo Meets Batman',
 'LEGO DC Comics Super Heroes: Batman: Be-Leaguered',
 'Batman vs. Robin',
 'The Lego Batman Movie',
 'Batman Unlimited: Animal Instincts',
 'Batman Unlimited: Monster Mayhem',
 'Batman: Bad Blood',
 'Batman and Friends',
 'Batman: The Killing Joke',
 'LEGO 

In [18]:
getCharacter("Batman", getMoviesWithGenre(16,batman_movies))

[('Batman: Gotham Knight', 'Kevin Conroy', 'Batman / Bruce Wayne (voice)'),
 ('Batman: Mask of the Phantasm',
  'Kevin Conroy',
  'Batman / Bruce Wayne (voice)'),
 ('Batman & Mr. Freeze: SubZero', 'Kevin Conroy', 'Bruce Wayne / Batman'),
 ('Batman Beyond: Return of the Joker',
  'Will Friedle',
  "Terence 'Terry' McGinnis / Batman"),
 ('Batman Beyond: Return of the Joker',
  'Kevin Conroy',
  'Bruce Wayne / Batman'),
 ("The Batman Superman Movie: World's Finest",
  'Kevin Conroy',
  'Batman / Bruce Wayne (voice)'),
 ('The Batman vs. Dracula', 'Rino Romano', 'Bruce Wayne / Batman'),
 ('Batman: Mystery of the Batwoman', 'Kevin Conroy', 'Bruce Wayne / Batman'),
 ('Superman/Batman: Public Enemies',
  'Kevin Conroy',
  'Batman / Bruce Wayne (voice)'),
 ('Batman: Under the Red Hood',
  'Bruce Greenwood',
  'Batman / Bruce Wayne (voice)'),
 ('Superman/Batman: Apocalypse', 'Kevin Conroy', 'Batman (voice)'),
 ('Batman Beyond: The Movie', 'Will Friedle', 'Terry McGinnis / Batman'),
 ('Batman Bey

In [0]:
movies_akira_as_director = getJobFrom("Director", getPerson(searchPerson("Akira Kurosawa")["results"][0]["id"])["movie_credits"]["crew"])

In [24]:
[(work["title"],work["popularity"]) for work in movies_akira]

[('Seven Samurai', 16.154),
 ('Rashomon', 10.607),
 ('The Hidden Fortress', 10.845),
 ('Throne of Blood', 9.095),
 ('Red Beard', 8.588),
 ('Ikiru', 11.991),
 ('Rhapsody in August', 2.342),
 ('Stray Dog', 7.18),
 ('I Live in Fear', 4.393),
 ('Ran', 8.982),
 ('Sanjuro', 6.838),
 ('The Bad Sleep Well', 7.59),
 ('Yojimbo', 9.009),
 ('The Idiot', 5.632),
 ('Kagemusha', 10.457),
 ('Drunken Angel', 6.454),
 ('Scandal', 4.115),
 ('High and Low', 8.822),
 ('Dreams', 7.869),
 ("Dodes'ka-den", 3.487),
 ('The Lower Depths', 4.189),
 ('Dersu Uzala', 8.02),
 ('Sanshiro Sugata', 3.679),
 ('Madadayo', 3.251),
 ("The Men Who Tread on the Tiger's Tail", 4.29),
 ('No Regrets for Our Youth', 2.188),
 ('The Quiet Duel', 4.315),
 ('Sanshiro Sugata Part Two', 1.034),
 ('One Wonderful Sunday', 4.714),
 ('The Most Beautiful', 3.04),
 ('Those Who Make Tomorrow', 1.4),
 ('Song of the Horse', 0.873)]